### 크롤링 도중 ip 차단을 당했고 실시간으로 ip를 랜덤하게 변환시키는 방법을 찾음<br> 그러나 Gamespy에서 제공하는 api로 id와 name을 찾음<br> 시간상의 이유나 모든 상황을 볼때 새로 steam spy에서 크롤링하는것이 더 좋은 판단이라고 생각되어 크롤링을 새로 시작

# 크롤링

In [6]:
import time
import pandas as pd
import requests
import pymysql
import json
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from selenium import webdriver

In [7]:
host_name = "127.0.0.1"
username = "root"
password = ""
database_name = "steam_test"

In [8]:
db = pymysql.connect(
    host = host_name,
    port=3306,
    user=username,
    passwd=password,
    db=database_name,
    charset='utf8'
)

## 테이블 생성

In [4]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
chromedriver = '/home/nincastle/chromedriver/chromedriver' # ubuntu
# chromedriver = 'D:\chromedriver_win32/chromedriver' # windows
tags_genre_query = []

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def driver_chrome(url, xpath_):
    driver_tags = webdriver.Chrome(chromedriver)
    driver_tags.get(url)
    print(type(xpath_))
    tags = driver_tags.find_element_by_xpath(xpath_)
    t_g = tags.find_elements_by_tag_name('a')
    driver_tags.quit
    return t_g
# 테그 변수 이름 크롤링
cro_tag = driver_chrome("https://steamdb.info/tags/","//div[@class='row list']")
for t in cro_tag:
    t = t.text.replace(" ","_")   
    tags_genre_query.append("tag_"+t+' INT(1)')


# 장르 변수 이름 크롤링
cro_gen = driver_chrome("https://steamdb.info/genres/","//div[@class='row']")
for g in cro_gen:
    g = g.text.replace(" ","_")
    tags_genre_query.append("genre_"+g+' INT(1)')

# driver_tags.quit
tag_genre_join = (", \n".join(tags_genre_query))

# SQL문 필터링
tag_genre_join = name_replace(tag_genre_join)
try: 
    cursor = db.cursor()
    SQL_QUERY = """
    # DROP DATABASE IF EXISTS 테이블이름;
    # CREATE DATABASE 테이블이름 DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

    USE steam_test;
    DROP TABLE IF EXISTS steam_test.data_merge;
    CREATE TABLE steam_test.data_merge (
        appid int(7),
        name varchar(100),
        owners int(10),
        owners_variance int(7),
        negative int(7),
        positive int(6),
        score_rank int(3),
        ccu int(6),
        players_2weeks int(8),
        players_forever_variance int(7),
        players_2weeks_variance int(7),
        userscore int(2),
        median_forever int(3),
        players_forever int(7),
        price int(7),
        average_2weeks int(5),
        developer varchar(300),
        median_2weeks int(4),
        publisher varchar(100),
        average_forever int(6),
        tag_Dystopian_ int(1),
        tag_Parody_ int(1),
        tag_VR_Only int(1),
        """+tag_genre_join+""",
        PRIMARY KEY(appid)
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin
    """

    create_sql=("".join(SQL_QUERY))
    print(create_sql)
    cursor.execute(create_sql)
    db.commit()
finally:
    print("ok!")
    pass

#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

### 초기 ID, name 데이터

In [5]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
with open("request_all.txt","r") as file:
    f = file.read()
    data = json.loads(f)
    for k,v in data.items():
        key = []
        value = []
        for k2,v2 in v.items():
            print(k2," : ",v2)
            if k2 == "score_rank":
                if v2 == "":
                    v2 = 0
            elif k2 == "developer" or k2 =="publisher":
                if v2 =="":
                    v2 = "null"
            elif k2 == "price":
                if v2 == "":
                    v2 == 999999                    
            key.append(k2)
            value.append(v2)
            
        key = tuple(key)
        key = ", ".join(key)
        value = tuple(value)
        sql ="""
            INSERT INTO steam_test.steamtable2 ({}) 
            VALUES 
            {};
            """.format(key, value)
        print(sql)
        cursor.execute(sql)
        db.commit()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

### 최종데이터 (크롤링 기간 2018-01-16~2018-01-19)

In [9]:
start_time = time.time()
#-----------------------------------------------------------------------------------------

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def select_number(start, last):
    get_data = """
    # select appid, name from steamtable2 LIMIT 2
    SELECT appid, name
    FROM steamtable2
    WHERE
        appid BETWEEN {} AND {};
    """.format(start, last)
    
    cursor = db.cursor()
    cursor.execute(get_data)
    row_names = cursor.fetchall()

    count = 0
    for s_num in row_names:
        print(s_num[0],s_num[1],count)
        count += 1  
        column_str=[]
        row_str=[]
        
        steamspy_url =('https://steamspy.com/api.php?request=appdetails&appid={}'.format(s_num[0]))
        appdetails_response = requests.get(steamspy_url)
        data = appdetails_response.json()
        
        for data_k, data_v in data.items():
            data_k = name_replace(data_k).replace(" ","_")  
            if data_k == 'tags':
                if data['tags'] == []:
                    pass
                else:
                    for tag_k ,tag_v in data['tags'].items():
                        tag_k = name_replace(tag_k).replace(" ","_")
                        column_str.append(("tag_{}").format(tag_k))
                        row_str.append(1)
                        
            elif data_k == "score_rank":
                if data_k == "":
                    column_str.append(data_k)
                    row_str.append(0)
                    
            elif data_k == "developer" or data_k =="publisher":
                if data_v =="":
                    column_str.append(data_k)
                    row_str.append("null")
                    
            elif data_k == "price":
                if data_v == "":
                    column_str.append(data_k)
                    row_str.append(999999)
                    
            else:
                column_str.append(data_k)
                row_str.append(data_v)
        try:
            cursor = db.cursor()
            sql = ("INSERT INTO steam_test.data_merge ("+(",".join(column_str)+") VALUES {}"\
                                                          .format(tuple(row_str))))
            cursor.execute(sql)
            db.commit()
        finally:
            pass    
#----------------------------------------------------------------------------------------


select_number(10, 999999)
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

# 데이터 정제

추가
* data_merge + steamtable2

In [57]:
copy_table = """
DROP TABLE IF EXISTS steam_test.merge_steam;
create table steam_test.merge_steam like steam_test.data_merge
"""
insert_data_merge = """
INSERT INTO steam_test.merge_steam 
SELECT * FROM steam_test.data_merge;
"""
delet_publisher = """
update merge_steam
set publisher = null
where publisher is not null;
"""
delet_developer = """
update merge_steam
set developer = null
where developer is not null;
"""

In [58]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
cursor = db.cursor()
cursor.execute(copy_table)
cursor.execute(insert_data_merge)
cursor.execute(delet_developer)
cursor.execute(delet_publisher)
db.commit()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

3.707911729812622 초(처리시간)


In [64]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
test_steamtable2 ="""
SELECT appid, score_rank, price, developer, publisher FROM steamtable2
"""

cursor.execute(test_steamtable2)
a_data = cursor.fetchall()
for i in a_data:
#     print("appid : {}, score_rank: {}, price: {}, developer: {}, publisher:{}".format(i[0], i[1], i[2], i[3], i[4]))
    try:
        update_data = """
        update merge_steam
        set score_rank = {}, price = {}, developer = "{}", publisher = "{}"
        where appid = {};
        """.format(i[1], i[2], i[3], i[4], i[0])
        print(update_data)
        cursor.execute(update_data)
    #     print(update_data)
    except:
        update_data = """
        update merge_steam
        set score_rank = {}, price = {}, developer = '{}', publisher = '{}'
        where appid = {};
        """.format(i[1], i[2], i[3], i[4], i[0])
        print(update_data)
        cursor.execute(update_data)
            
db.commit()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))


        update merge_steam
        set score_rank = 97, price = 999, developer = "Valve", publisher = "Valve"
        where appid = 10;
        

        update merge_steam
        set score_rank = 56, price = 499, developer = "Valve", publisher = "Valve"
        where appid = 20;
        

        update merge_steam
        set score_rank = 74, price = 499, developer = "Valve", publisher = "Valve"
        where appid = 30;
        

        update merge_steam
        set score_rank = 53, price = 499, developer = "Valve", publisher = "Valve"
        where appid = 40;
        

        update merge_steam
        set score_rank = 89, price = 499, developer = "Gearbox Software", publisher = "Valve"
        where appid = 50;
        

        update merge_steam
        set score_rank = 46, price = 499, developer = "Valve", publisher = "Valve"
        where appid = 60;
        

        update merge_steam
        set score_rank = 95, price = 999, developer = "Valve", publisher = "Valve"
  


        update merge_steam
        set score_rank = 67, price = 999, developer = "Lazy 8 Studios", publisher = "null"
        where appid = 26500;
        

        update merge_steam
        set score_rank = 89, price = 1499, developer = "Number None", publisher = "Number None"
        where appid = 26800;
        

        update merge_steam
        set score_rank = 59, price = 1999, developer = "Kloonigames", publisher = "Kloonigames"
        where appid = 26900;
        

        update merge_steam
        set score_rank = 36, price = 999, developer = "Tale of Tales", publisher = "Tale of Tales"
        where appid = 27000;
        

        update merge_steam
        set score_rank = 15, price = 499, developer = "Tale of Tales", publisher = "Tale of Tales"
        where appid = 27020;
        

        update merge_steam
        set score_rank = 11, price = 699, developer = "Tale of Tales", publisher = "Tale of Tales"
        where appid = 27050;
        

        update merge_st


        update merge_steam
        set score_rank = 0, price = 0, developer = "null", publisher = "null"
        where appid = 71300;
        

        update merge_steam
        set score_rank = 0, price = 0, developer = "null", publisher = "null"
        where appid = 71310;
        

        update merge_steam
        set score_rank = 0, price = 0, developer = "null", publisher = "null"
        where appid = 71320;
        

        update merge_steam
        set score_rank = 81, price = 1999, developer = "Devil's Details", publisher = "SEGA"
        where appid = 71340;
        

        update merge_steam
        set score_rank = 0, price = 1499, developer = "SEGA", publisher = "SEGA"
        where appid = 71390;
        

        update merge_steam
        set score_rank = 64, price = 999, developer = "Eyebrow Interactive", publisher = "Eyebrow Interactive"
        where appid = 72000;
        

        update merge_steam
        set score_rank = 49, price = 0, developer = "Gian


        update merge_steam
        set score_rank = 13, price = 1999, developer = "Reality Pump", publisher = "Topware Interactive"
        where appid = 224900;
        

        update merge_steam
        set score_rank = 34, price = 699, developer = "Crystal Dynamics", publisher = "Square Enix"
        where appid = 224920;
        

        update merge_steam
        set score_rank = 36, price = 699, developer = "Crystal Dynamics", publisher = "Square Enix"
        where appid = 224940;
        

        update merge_steam
        set score_rank = 56, price = 699, developer = "Core Design", publisher = "Square Enix"
        where appid = 224960;
        

        update merge_steam
        set score_rank = 67, price = 699, developer = "Core Design", publisher = "Square Enix"
        where appid = 224980;
        

        update merge_steam
        set score_rank = 34, price = 699, developer = "Core Design", publisher = "Square Enix"
        where appid = 225000;
        

       


        update merge_steam
        set score_rank = 28, price = 399, developer = "DigitalDNA Games LLC", publisher = "DigitalDNA Games LLC"
        where appid = 253430;
        

        update merge_steam
        set score_rank = 0, price = 899, developer = "Playstos Entertainment", publisher = "Inky Mind UK"
        where appid = 253470;
        

        update merge_steam
        set score_rank = 18, price = 0, developer = "ESTsoft Corp.", publisher = "ESTsoft Inc."
        where appid = 253490;
        

        update merge_steam
        set score_rank = 6, price = 0, developer = "WhiteMoon Dreams", publisher = "Privateer Press Interactive"
        where appid = 253510;
        

        update merge_steam
        set score_rank = 3, price = 499, developer = "Lucky Frame", publisher = "Blazing Griffin Ltd."
        where appid = 253570;
        

        update merge_steam
        set score_rank = 33, price = 999, developer = "Final Boss Entertainment", publisher = "Final Boss 

...


        update merge_steam
        set score_rank = 95, price = 299, developer = "一次元创作组", publisher = "一次元创作组"
        where appid = 630060;
        

        update merge_steam
        set score_rank = 0, price = 399, developer = "Bartoš Studio", publisher = "Bartoš Studio"
        where appid = 630070;
        

        update merge_steam
        set score_rank = 0, price = 999, developer = "Oddly Shaped Pixels", publisher = "Digital Tribe"
        where appid = 630160;
        

        update merge_steam
        set score_rank = 0, price = 999, developer = "RSK Entertainment", publisher = "Strategy First"
        where appid = 630170;
        

        update merge_steam
        set score_rank = 0, price = 999, developer = "Mad Head Games", publisher = "Big Fish Games"
        where appid = 630240;
        

        update merge_steam
        set score_rank = 61, price = 1999, developer = "WayForward", publisher = "WayForward"
        where appid = 630310;
        

        updat


        update merge_steam
        set score_rank = 3, price = 0, developer = "SOFTON ENTERTAINMENT", publisher = "SOFTON ENTERTAINMENT"
        where appid = 658510;
        

        update merge_steam
        set score_rank = 95, price = 1499, developer = "Frontwing", publisher = "Frontwing USA"
        where appid = 658530;
        

        update merge_steam
        set score_rank = 92, price = 99, developer = "Quiet River", publisher = "Quiet River"
        where appid = 658560;
        

        update merge_steam
        set score_rank = 67, price = 1499, developer = "Drift Physics Crew", publisher = "Drift Physics Crew"
        where appid = 658570;
        

        update merge_steam
        set score_rank = 31, price = 59, developer = "Tero Lunkka, Mikko Seppälä", publisher = "Tero Lunkka"
        where appid = 658600;
        

        update merge_steam
        set score_rank = 2, price = 119, developer = "singi", publisher = "singi"
        where appid = 658610;
      


        update merge_steam
        set score_rank = 44, price = 299, developer = "Clarus Victoria", publisher = "Clarus Victoria"
        where appid = 686900;
        

        update merge_steam
        set score_rank = 0, price = 499, developer = "Night Whale", publisher = "Night Whale"
        where appid = 686910;
        

        update merge_steam
        set score_rank = 0, price = 0, developer = "Christopher Howard", publisher = "Christopher Howard"
        where appid = 686940;
        

        update merge_steam
        set score_rank = 0, price = 0, developer = "NMY Mixed-Reality Communication GmbH", publisher = "B. Braun"
        where appid = 686950;
        

        update merge_steam
        set score_rank = 81, price = 1799, developer = "Cherrymochi", publisher = "Square Enix"
        where appid = 687260;
        

        update merge_steam
        set score_rank = 15, price = 0, developer = "Dark Polygon Ltda., Jalomot Ltda.", publisher = "Dark Polygon Ltda."
  


        update merge_steam
        set score_rank = 9, price = 199, developer = "Vyacheslav Shilikhin", publisher = "Vyacheslav Shilikhin"
        where appid = 715980;
        

        update merge_steam
        set score_rank = 100, price = 99, developer = "Smokey Moose Games", publisher = "Smokey Moose Games"
        where appid = 716070;
        

        update merge_steam
        set score_rank = 5, price = 99, developer = "lalalaZero, Urbanoff", publisher = "Cisamid Inc."
        where appid = 716110;
        

        update merge_steam
        set score_rank = 0, price = 199, developer = "EVGENII MAKHNEV", publisher = "EVGENII MAKHNEV"
        where appid = 716140;
        

        update merge_steam
        set score_rank = 0, price = 200, developer = "Anxious Neck Games", publisher = "Anxious Neck Games"
        where appid = 716150;
        

        update merge_steam
        set score_rank = 74, price = 0, developer = "Chainsawesome Games", publisher = "Chainsawesome G


        update merge_steam
        set score_rank = 0, price = 99, developer = "DIGITAL MISTAKE", publisher = "DIGITAL MISTAKE"
        where appid = 762460;
        

        update merge_steam
        set score_rank = 85, price = 0, developer = "Wooden Shoes Games", publisher = "Wooden Shoes Games"
        where appid = 763230;
        

        update merge_steam
        set score_rank = 0, price = 99, developer = "BadCat Studios", publisher = "BadCat Studios"
        where appid = 763400;
        

        update merge_steam
        set score_rank = 56, price = 0, developer = "Doborog Games", publisher = "Doborog Games"
        where appid = 763410;
        

        update merge_steam
        set score_rank = 36, price = 799, developer = "Fantahorn Studio", publisher = "Vive Studios"
        where appid = 763430;
        

        update merge_steam
        set score_rank = 77, price = 0, developer = "Rocket Tiger Studios", publisher = "Rocket Tiger Studios"
        where appid =

teg, genre null to 0

In [ ]:
# 변수명
c_name = """
SELECT COLUMN_NAME
FROM information_schema.columns 
WHERE 
    table_schema = 'steam_test' 
    AND 
    table_name = 'merge_steam';
"""
cursor.execute(c_name)
u_t_g = cursor.fetchall()
# tag range 20 ~ 362
# genre range 363 ~ 387
t_g_list = []
for a in u_t_g:
    t_g_list.append(list(a))
b = t_g_list[20:387]

print(",".join(b))

# """
# UPDATE merge_steam SET [컬럼명] = ISNULL([컬럼명], '0')
# """